<a href="https://colab.research.google.com/github/Bigchicken166/Daphne/blob/main/texto_a_musica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convierte texto a música ✍ ➡ 🎹🎵🔊**

Notebook para crear música con Inteligencia artificial.

# Sección nueva

In [ ]:
#@title **Iniciar el algoritmo**

import subprocess, time
print("Despertando a la inteligencia artificial...")
start_time = time.time()
all_process = [
    ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
    ['pip', 'install', '-U', 'sentence-transformers'],
    ['pip', 'install', 'httpx'],
]
for process in all_process:
    running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')

end_time = time.time()
print(f"IA iniciada en {end_time-start_time:.0f} segundos")

Despertando a la inteligencia artificial...
IA iniciada en 208 segundos


In [ ]:
#@title **Definir métodos y precálculos** { display-mode: "form" }

import numpy as np
from sentence_transformers import SentenceTransformer
minilm = SentenceTransformer('all-MiniLM-L6-v2')

mubert_tags_string = 'meditation,bells,drums,piano,keys,classical,orchestral,hiphop,jazz,om,zen,yoga'
mubert_tags = np.array(mubert_tags_string.split(','))
mubert_tags_embeddings = minilm.encode(mubert_tags)

from IPython.display import Audio, display
import httpx
import json

def get_track_by_tags(tags, pat, duration, maxit=20, autoplay=False, loop=False):
  if loop:
    mode = "loop"
  else:
    mode = "track"
  r = httpx.post('https://api-b2b.mubert.com/v2/RecordTrackTTM',
      json={
          "method":"RecordTrackTTM",
          "params": {
              "pat": pat,
              "duration": duration,
              "tags": tags,
              "mode": mode
          }
      })

  rdata = json.loads(r.text)
  assert rdata['status'] == 1, rdata['error']['text']
  trackurl = rdata['data']['tasks'][0]['download_link']

  print('Generando música ', end='')
  for i in range(maxit):
      r = httpx.get(trackurl)
      if r.status_code == 200:
          display(Audio(trackurl, autoplay=autoplay))
          break
      time.sleep(1)
      print('.', end='')

def find_similar(em, embeddings, method='cosine'):
    scores = []
    for ref in embeddings:
        if method == 'cosine':
            scores.append(1 - np.dot(ref, em)/(np.linalg.norm(ref)*np.linalg.norm(em)))
        if method == 'norm':
            scores.append(np.linalg.norm(ref - em))
    return np.array(scores), np.argsort(scores)

def get_tags_for_prompts(prompts, top_n=3, debug=False):
    prompts_embeddings = minilm.encode(prompts)
    ret = []
    for i, pe in enumerate(prompts_embeddings):
        scores, idxs = find_similar(pe, mubert_tags_embeddings)
        top_tags = mubert_tags[idxs[:top_n]]
        top_prob = 1 - scores[idxs[:top_n]]
        if debug:
            print(f"Prompt: {prompts[i]}\nTags: {', '.join(top_tags)}\nScores: {top_prob}\n\n\n")
        ret.append((prompts[i], list(top_tags)))
    return ret

In [ ]:
#@markdown **Obtener un token de acceso personal y definir métodos API**
email = ""

r = httpx.post('https://api-b2b.mubert.com/v2/GetServiceAccess',
    json={
        "method":"GetServiceAccess",
        "params": {
            "email": email,
            "license":"ttmmubertlicense#f0acYBenRcfeFpNT4wpYGaTQIyDI4mJGv5MfIhBFz97NXDwDNFHmMRsBSzmGsJwbTpP1A6i07AXcIeAHo5",
            "token":"4951f6428e83172a4f39de05d5b3ab10d58560b8",
            "mode": "loop"
        }
    })

rdata = json.loads(r.text)
assert rdata['status'] == 1, "probably incorrect e-mail"
pat = rdata['data']['pat']
print(f'Got token: {pat}')

In [ ]:
#@title **Crea música 🎵** { vertical-output: true, form-width: "50%", display-mode: "form" }
#@markdown Luego de ver ambas versiones de Bosquejos de la india, te dejamos las siguientes preguntas para crear una pieza musical a través de Inteligencia Artificial:

#@markdown ¿A qué género musical te gustaría que se vincule tu creación?
prompt = "" #@param {type:"string"}
#@markdown ¿Dónde te imaginas que sucede la escena que muestra la obra?
prompt = "" #@param {type:"string"}
#@markdown Si tuvieras que asociar la obra Bosquejos de la India (textura) a algún aroma, ¿Cuál sería?
prompt = "" #@param {type:"string"}
#@markdown Si la obra Bosquejos de la India (blanca) tuviera color, ¿Que colores tendría?
prompt = "" #@param {type:"string"}
#@markdown ¿Qué emoción te produce la obra Bosquejos de la India?
prompt = "" #@param {type:"string"}
prompt = "om,zen,yoga,meditation"
duration = 30
loop = False

def generate_track_by_prompt(prompt, duration, loop=False):
  _, tags = get_tags_for_prompts([prompt,])[0]
  try:
    get_track_by_tags(tags, pat, duration, autoplay=True, loop=loop)
  except Exception as e:
    print(str(e))
  print('\n')




generate_track_by_prompt(prompt, duration, loop)